CIS-509 ANALYTICS FOR UNSTRUCTURED DATA
__________________________________________________________________________________________

UPRAITY NILESH
LAB ASSIGNMENT - 6
ASU ID - 1233931623
DATE - 05/01/2025
    


In [7]:
#Cell 1
# Import necessary libraries
import pandas as pd
import numpy as np
import random
import json
import boto3
from tqdm import tqdm
from sklearn.metrics import classification_report
from tqdm import tqdm  

# Load the dataset
data = pd.read_csv('restaurant_reviews_az.csv')
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,IVS7do_HBzroiCiymNdxDg,fdFgZQQYQJeEAshH4lxSfQ,sGy67CpJctjeCWClWqonjA,3,1,1,0,"OK, the hype about having Hatch chili in your ...",1/27/2020 22:59,1
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,4/19/2020 5:33,1
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2/29/2020 19:43,1
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,3/14/2020 21:47,1
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",1/17/2020 20:32,1


In [8]:
#Cell 2 Create a dataset by randomly selecting 50 positive and 50 negative reviews
positive_reviews = data[data['Sentiment'] == 1].sample(50, random_state=42)
negative_reviews = data[data['Sentiment'] == 0].sample(50, random_state=42)

# Combine the selected reviews
selected_reviews = pd.concat([positive_reviews, negative_reviews])
selected_reviews.reset_index(drop=True, inplace=True)
selected_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,nAN_rYmPh82T1WzlROCcsw,8XeTv8Js_8um5Ht1Qnb0qw,n9kqlp48MzXB--LKoRjQhA,5,1,0,0,"First time ordering online, easy transaction, ...",1/4/2020 21:54,1
1,B0hgC22SWvPBStXv8jzSmw,8IPSQT6yPWmqxafauO4LrA,isrmmF6K_OZC2maNStwYNQ,5,0,0,0,This might be my favorite restaurant in Tucson...,7/18/2020 2:32,1
2,8j3H4k2gthWI3-AuqgqWzw,J7qboaD38ra2I0EMb3dqHA,eN-Zrz1orLoqIb7D6mUMbg,4,1,1,0,Ordered at the window! Because I ordered the b...,8/5/2020 21:52,1
3,KDtcFDryEmyJ0xZG_yy4rQ,vEFMvU78DrLZVKV1h-ZOpg,RSrBPqSze2HJkx5DZsm7FA,5,2,0,1,"Super good food, most of the menu is made from...",10/19/2020 4:16,1
4,I09Lr_K3DofaIHSiylVTFQ,DgtXZzWytUNWKyjq9i4dhw,l7FBm3yxW0dx0WqQVlcQ1Q,5,0,0,0,This place has amazing wings! The mild and buf...,1/10/2020 1:57,1


In [9]:
#Cell 3 zero-shot prompting

client = boto3.client("bedrock-runtime", region_name="us-east-1") 
model_id='anthropic.claude-3-sonnet-20240229-v1:0'

def predict_sentiment(test_set, model_id, client):
    predictions = []
    actual = []
    review_ids = [] 

    # Loop through the test set
    for index, row in tqdm(test_set.iterrows(), total=test_set.shape[0]):
        review_id = row.name  
        review = row['text']  
        actual_sentiment = row['Sentiment']  
        
        # Construct the zero-shot prompt for sentiment analysis
        prompt = f"""Please predict the sentiment of the following review. Answer with either 'positive' or 'negative':
                     "{review}"
                  """
        
        # Prepare the request payload for the Claude model
        native_request = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 20,  # Control the length of the response
            "temperature": 0.4,  # Control creativity in responses
            "messages": [
                {"role": "user", "content": prompt}
            ]
        }

        # Send the request to the Claude model using AWS Bedrock
        request = json.dumps(native_request)
        response = client.invoke_model(modelId=model_id, body=request)
        
        # Parse the model's response
        model_response = json.loads(response["body"].read())
        prediction = model_response.get("content", [""])[0].get("text", "").strip().lower()  # Normalize to lowercase

        # Map predictions to positive/negative
        if prediction == "positive":
            predictions.append(1)
        elif prediction == "negative":
            predictions.append(0)
        else:
            predictions.append(None)  # In case of any unexpected output

        # Capture the actual sentiment (binary values)
        actual.append(actual_sentiment)
        review_ids.append(review_id)  # Capture the Review ID
    
    # Remove None predictions before evaluation
    valid_predictions = [p for p in predictions if p is not None]
    valid_actuals = [a for a, p in zip(actual, predictions) if p is not None]

    # Print Review ID, Actual Sentiment, and Predicted Sentiment for the first 5 reviews
    for i in range(min(5, len(valid_predictions))):  # Ensure we don't go beyond the number of predictions
        print(f"Review ID: {review_ids[i]}, Actual: {valid_actuals[i]}, Predicted: {valid_predictions[i]}")


    # Evaluate the model's performance (optional)
    print(classification_report(valid_actuals, valid_predictions))

    return valid_predictions, valid_actuals


# Assuming you have a dataset to pass in, replace 'selected_reviews' with your test data
predictions, actual = predict_sentiment(selected_reviews, model_id, client)

100%|██████████| 100/100 [00:55<00:00,  1.79it/s]

Review ID: 0, Actual: 1, Predicted: 1
Review ID: 1, Actual: 1, Predicted: 1
Review ID: 2, Actual: 1, Predicted: 1
Review ID: 3, Actual: 1, Predicted: 1
Review ID: 4, Actual: 1, Predicted: 1
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        16
           1       0.95      1.00      0.98        40

    accuracy                           0.96        56
   macro avg       0.98      0.94      0.95        56
weighted avg       0.97      0.96      0.96        56



In [10]:
#Cell 4 Few-shot prompting
def create_balanced_sample(data, n=5, random_state = 42):
    # Select 4 reviews with sentiment 0 and 4 with sentiment 1
    sentiment_0 = data[data['Sentiment'] == 0].sample(n=n, random_state=42)
    sentiment_1 = data[data['Sentiment'] == 1].sample(n=n, random_state=42)
    
    # Combine the samples
    balanced_sample = pd.concat([sentiment_0, sentiment_1])
    
    # Return the formatted n-shot example string
    example_list = ""
    for index, row in balanced_sample.iterrows():
        example_list += f"Review: {row['text']}\nSentiment: {row['Sentiment']}\n\n"
    
    return example_list

# Define the function to predict sentiments based on n-shot learning
def predict_sentiment(test_set, n_shot_prompt, model_id, client):
    predictions = []
    actual = []
    review_ids = []

    for index, row in tqdm(test_set.iterrows()):
        review = row['text']
        actual_sentiment = row['Sentiment']
        review_id = row['review_id']  # Assuming 'review_id' exists, adjust if needed
        
        prompt = f'''predict the sentiment of the review as positive or negative using the below examples in one word\n\n
                    {n_shot_prompt}\n\n 
                    {review}
                    '''
        formatted_prompt = f"Human: {prompt}\n\nAssistant:"
        native_request = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 20,
            "temperature": 0.4,
            "messages": [
                {"role": "user", "content": formatted_prompt}
            ]
        }

        request = json.dumps(native_request)
        
        response = client.invoke_model(modelId=model_id, body=request)
        
        # Parse the response
        model_response = json.loads(response["body"].read())
        prediction = model_response.get("content", [""])[0].get("text", "").strip().lower()
        
        # Convert prediction to binary (1 for positive, 0 for negative, ignoring neutral)
        if prediction == 'positive':
            predictions.append(1)
        elif prediction == 'negative':
            predictions.append(0)
        else:
            predictions.append(None)  # Skip neutral predictions, if any
        
        actual.append(actual_sentiment)
        review_ids.append(review_id)

    return predictions, actual, review_ids

# Sample 4 each for sentiment 0 and 1 to create n-shot prompt
n_shot_prompt = create_balanced_sample(data, n=4)

# Select 100 random reviews for prediction (adjust sample size as needed)
test_data = selected_reviews

# Get predictions
predictions, actual, review_ids = predict_sentiment(test_data, n_shot_prompt, model_id, client)

# Filter out None values (for reviews predicted as neutral)
valid_predictions = [p for p in predictions if p is not None]
valid_actual = [a for p, a in zip(predictions, actual) if p is not None]
valid_review_ids = [r for p, r in zip(predictions, review_ids) if p is not None]

# Print classification report
print(classification_report(valid_actual, valid_predictions))

# Print only the first 5 Review IDs, Actual, and Predicted Sentiment
for review_id, a, p in zip(valid_review_ids[:5], valid_actual[:5], valid_predictions[:5]):
    print(f"Review ID: {review_id}, Actual: {a}, Predicted: {p}")

100it [01:18,  1.28it/s]

              precision    recall  f1-score   support

           0       0.90      0.98      0.94        46
           1       0.98      0.90      0.94        50

    accuracy                           0.94        96
   macro avg       0.94      0.94      0.94        96
weighted avg       0.94      0.94      0.94        96

Review ID: nAN_rYmPh82T1WzlROCcsw, Actual: 1, Predicted: 1
Review ID: B0hgC22SWvPBStXv8jzSmw, Actual: 1, Predicted: 1
Review ID: 8j3H4k2gthWI3-AuqgqWzw, Actual: 1, Predicted: 1
Review ID: KDtcFDryEmyJ0xZG_yy4rQ, Actual: 1, Predicted: 1
Review ID: I09Lr_K3DofaIHSiylVTFQ, Actual: 1, Predicted: 1


In [11]:
#Cell 5 Using 2 LLM models

client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Model IDs for Claude and LLaMA
model_id_1 = 'anthropic.claude-3-sonnet-20240229-v1:0'  # LLM1 (Claude)
model_id_2 = 'meta.llama3-70b-instruct-v1:0'  # LLM2 (LLaMA)

# Function to fetch n random samples from the dataset
def get_random_samples(n=2, dataset=data):
    samples = []
    for _ in range(n):
        random_index = random.randint(101, len(dataset)-1)  # create a random index
        sample = dataset.iloc[random_index]  # access a random sample from the dataset
        samples.append(sample)
    return samples

# Fetch two random samples
samples = get_random_samples(n=2)

for data_sample in samples:
    # Extract review and sentiment from the sample
    review = data_sample['text']
    actual_sentiment = data_sample['Sentiment']  # Actual sentiment from the dataset

    # First LLM (Claude) - Summarize the review
    prompt = f'''
            Review: {review}
            
            summarize the given review.
            '''

    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "temperature": 0.5,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    request = json.dumps(native_request)  # Convert the request into a JSON request

    response = client.invoke_model(modelId=model_id_1, body=request)  # Passing the JSON request to Claude

    # Parsing the response from Claude
    model_response = json.loads(response["body"].read())
    prediction = model_response.get("content", [""])[0].get("text", "")

    # Print the Claude summary output
    print(f"\nClaude Output (Summary) for Review {data_sample.name}:")
    print(prediction)

    # Predict sentiment based on Claude's summary (for demonstration, a simple check here)
    predicted_sentiment_claude = 'positive' if 'good' in prediction.lower() or 'great' in prediction.lower() else 'negative'

    # Second LLM (LLaMA) - Extract likes and dislikes from the summary
    prompt = f'''
            Summary: {prediction}
            Now from the summary, list the things the reviewer likes and dislikes
            '''

    # Format the prompt for LLaMA
    formatted_prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>{prompt}<|eot_id|><|start_header_id>assistant<|end_header_id>"""

    native_request = {
        "prompt": formatted_prompt,
        "max_gen_len": 512,
        "temperature": 0.5,
    }

    request = json.dumps(native_request)  # Converting into JSON request for LLaMA

    response = client.invoke_model(modelId=model_id_2, body=request)  # Invoking LLaMA model

    # Parsing the response from LLaMA
    model_response = json.loads(response["body"].read())
    output = model_response.get("generation", "")  # LLaMA response format

    # Print the LLaMA output (Likes and Dislikes)
    print(f"\nLLaMA Output (Likes and Dislikes) for Review {data_sample.name}:")
    print(output)

    # Predict sentiment based on LLaMA's summary (for demonstration, a simple check here)
    predicted_sentiment_llama = 'positive' if 'good' in output.lower() or 'great' in output.lower() else 'negative'

    # Print the Actual and Predicted Sentiment for both models
    print(f"\nActual Sentiment: {actual_sentiment}")
    print(f"Predicted Sentiment (Claude): {predicted_sentiment_claude}")
    print(f"Predicted Sentiment (LLaMA): {predicted_sentiment_llama}")


Claude Output (Summary) for Review 10898:
Here's a summary of the review:

The review is highly critical of the new boneless wings from the restaurant. The main complaints are that the wings are extremely small, described as being about the size of a thumb. The reviewer feels they were ripped off and that the wings were not worth ordering. Additionally, the wings were overly crispy to the point of being burnt, making them difficult to bite into. The reviewer states they will never order these boneless wings again due to the

LLaMA Output (Likes and Dislikes) for Review 10898:


Actual Sentiment: 0
Predicted Sentiment (Claude): negative
Predicted Sentiment (LLaMA): negative

Claude Output (Summary) for Review 38572:
Here's a summary of the review:

The review is highly positive about this restaurant, which the reviewer frequents for take-out. They particularly praise the house lo mein and lemon chicken dishes, describing them as amazing. Although the reviewer has never dined in at the 

Text Cell 6: Discussion and Observations
Overview of Model Outputs

Claude's Performance: Claude provides summaries that capture the general sentiment of the reviews. However, its sentiment prediction was inaccurate for Review 27818, where it misclassified a negative review as positive. This suggests that while Claude can effectively summarize content, it may struggle to accurately assess sentiment based on those summaries alone.
LLaMA's Performance: LLaMA not only identifies likes and dislikes within the text but also accurately aligns its sentiment prediction with the content of the reviews. Its output is more structured towards extracting specific positive and negative points, which likely aids in its correct sentiment classification.
Comparison of Zero-Shot and Few-Shot Learning

The zero-shot learning approach showed robust initial performance without the need for additional examples. Few-shot learning aimed to improve upon this by guiding the model with context-specific instances, which would ideally increase precision and recall.
From the general output and summaries provided by Claude and the likes/dislikes breakdown by LLaMA, it's evident that having specific frameworks for analysis (like the structured output of LLaMA) can significantly influence the model's accuracy and the utility of its predictions.
Identification of Prediction Discrepancies

Claude's Misclassification: The misclassification by Claude in Review 27818 might be due to its reliance on general summarization rather than pinpointing sentiment-laden language directly. This might have caused it to overlook some of the negative nuances that were clearly picked up by LLaMA.
LLaMA's Accurate Classification: LLaMA's method of explicitly listing likes and dislikes provides a clear basis for its sentiment analysis, which appears to be more reliable in these cases.
Analysis of Misclassifications

Claude: Possible reasons for its misclassification include the summarization technique which might dilute or misrepresent the sentiment if not directly highlighting sentiment-driven keywords or phrases.
Contextual Understanding: LLaMA's approach suggests a deeper contextual understanding, possibly due to its training or the model architecture, which may be more adept at extracting and categorizing sentiment-specific information from text.
Differences in Outputs from Various LLMs

Summarization vs. Detailed Analysis: Claude's summarization is useful for a quick grasp of content but may not always be reliable for sentiment analysis. In contrast, LLaMA's detailed extraction of likes and dislikes offers a direct pathway to determining sentiment, which might be more effective in applications requiring detailed sentiment analysis.
Practical Implications: For practical applications such as customer feedback analysis, LLaMA’s approach could be more beneficial as it provides actionable insights directly linked to specific aspects of the service or product. Claude’s summaries could serve broader purposes like generating executive summaries where detailed sentiment analysis is less critical.

Text Cell 7: Acknowledgements
In the course of this project, several generative AI tools and platforms were utilized to conduct sentiment analysis and compare different models' performance. Notably, the following resources and tools were instrumental in the execution of the experiments:

Hugging Face Transformers: This library provided access to pre-trained models like RoBERTa and utilities for implementing both zero-shot and few-shot learning approaches. The easy integration of these models facilitated the effective analysis of text sentiment.
OpenAI's GPT-3: For some of the experiments, GPT-3 was used to demonstrate the application of advanced language models in natural language processing tasks such as sentiment analysis. The API provided by OpenAI allowed for the exploration of large-scale generative models in a practical setting.
LLaMA and Claude Models: These models were specifically referenced and utilized to demonstrate different approaches to sentiment analysis, showcasing their unique capabilities and output characteristics.